In [1]:
## 基本操作

from peewee import *

# SQLite database using WAL journal mode and 64MB cache.
sqlite_db = SqliteDatabase('/path/to/app.db', pragmas={
    'journal_mode': 'wal',
    'cache_size': -1024 * 64})


In [4]:
## SQLite的扩展
from playhouse.sqlite_ext import SqliteExtDatabase

# Use SQLite (will register a REGEXP function and set busy timeout to 3s).
db = SqliteExtDatabase('people.db',
                       regexp_function=True,
                       timeout = 4,
                       pragmas = {
                           'journal_mode': 'wal',
                       } )


In [5]:
db.pragma('cache_size', -1024 *64)

In [16]:
print('cache_size:', db.cache_size)
print('foreign_keys:', db.foreign_keys)
print('journal_mode:', db.journal_mode)
print('page_size:', db.page_size)
print(db._table_functions)

cache_size: -65536
foreign_keys: 0
journal_mode: wal
page_size: 4096
[]


In [15]:
db.pragma('foreign_keys', 0, permanent= True)

In [17]:
#用户自定义函数部分没有看懂，装饰器的用法也忘记了

In [22]:
# 在程序运行时设置数据库的三种方法
#1 
db_1 = SqliteDatabase('my_app.db', pragmas = {'journal_mode':'wal'})

#2
db_2 = SqliteDatabase(None)

db_2.init('de_filename.db', pragmas={'journal_mode':'wal'})

#3
db_3 = DatabaseProxy()

db_3.initialize(db_1)


In [24]:
db_1.connect()

OperationalError: Connection already opened.

In [25]:
db_1.connect(reuse_if_open=True)

False

In [27]:
db_1.close()

False

In [28]:
db_2.is_closed

<bound method Database.is_closed of <peewee.SqliteDatabase object at 0x00000186F22BAB80>>

In [30]:
db_2.connect(reuse_if_open=True)

False

In [35]:
db_2.is_closed()

True

In [34]:
with db_2:
    print(db.is_closed())

False


In [32]:
db_2.close()

True

In [39]:
with db_1.connection_context():
    print(db_1.is_connection_usable())
    print(db_1.is_closed())

True
False


In [40]:
# 等下认真学习下装饰器的玩法，这里基本都在用

In [41]:
from playhouse.pool import PooledSqliteDatabase

db = PooledSqliteDatabase(
    'my_app.db', 
    max_connections= 30, # 最大连接数
    stale_timeout= 300, # 过期时间
    pragmas = {  # 其他参数
        'journal_mode': 'wal'
    })

class BaseModel(Model):
    class Meta:
        database = db

In [45]:
import unittest
from peewee import *

test_db = SqliteDatabase(':memory:')

class User(BaseModel):
    pass

class Tweet(BaseModel):
    pass

class EventLog(BaseModel):
    pass

class Relationship(BaseModel):
    pass

MODELS = [User, Tweet, EventLog, Relationship]

class BaseTestCase(unittest.TestCase):
    def setup(self):
        test_db.bind(# 3个参数
            MODELS,  # 准备绑定的Models
            bind_refs=False,  # 绑定引用的模型
            bind_backrefs=False) # 绑定反向引用的模型
        test_db.connect() 
        test_db.create_tables(MODELS)

    def tearDown(self):
        test_db.drop_tables(MODELS)
        test_db.close()

test_db.bind()

x = BaseTestCase()
x.setup()
x.tearDown()




In [46]:
# 与flask框架完美集成

from flask import Flask
from peewee import *

database = SqliteDatabase('my_app.db')
app = Flask(__name__)

# This hook ensures that a connection is opened to handle any queries
# generated by the request.
@app.before_request
def _db_connect():
    database.connect()

# This hook ensures that the connection is closed when we've finished
# processing the request.
@app.teardown_request
def _db_close(exc):
    if not database.is_closed():
        database.close()